In [4]:
from llama_hub.tools.waii import WaiiToolSpec

waii_tool = WaiiToolSpec(
    url="https://tweakit.waii.ai/api/",
    # API Key of Waii (not OpenAI API key)
    api_key="3a44......",
    # Which database you want to use, you need add the db connection to Waii first
    database_key="snowflake://..."
)

In [8]:
from llama_index import VectorStoreIndex

# Use as Data Loader, load data to index and query it
documents = waii_tool.load_data('Get all tables with their number of columns')
index = VectorStoreIndex.from_documents(documents).as_query_engine()

index.query('Which table contains most columns, tell me top 5 tables with number of columns?').response

"The table 'COLUMNS' contains the most columns. The top 5 tables with the number of columns are 'COLUMNS' with 43 columns, 'TABLES' with 25 columns, and the remaining tables have fewer than 25 columns."

In [23]:
# Use as tool, initialize it
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

agent = OpenAIAgent.from_tools(waii_tool.to_tool_list(), llm=OpenAI(model='gpt-4-1106-preview'), verbose=False)

In [24]:
# Ask simple questions
print(agent.chat("Give me top 3 countries with the most number of car factory"))

The top 3 countries with the highest number of car factories are Japan, Germany, and the USA.


In [25]:
print(agent.chat("What are the car factories of these countries"))

Here are the car factories in the top 3 countries with the most number of car factories:

- **Japan**: Nissan, Honda, Mazda, Subaru, and Toyota.
- **Germany**: Volkswagen, BMW, Daimler Benz, and Opel.
- **USA**: AMC, GM, Ford, and Chrysler.


In [26]:
# Do performance analysis
print(agent.chat("Give me top 3 longest running queries, and their duration."))

The top 3 longest running queries and their durations are:

1. Query ID: 01b0683d-0001-e41d-0022-ba8700baf82a, with a duration of 61,034 milliseconds.
2. Query ID: 01b07576-0001-e5c4-0022-ba8700bc1d62, with a duration of 33,450 milliseconds.
3. Query ID: 01b06494-0001-e50e-0022-ba8700bad9a2, with a duration of 25,301 milliseconds.


In [27]:
print(agent.chat("analyze the 2nd-longest running query"))

The analysis of the second-longest running query reveals the following:

**Summary**:
- The most time-consuming part of the query is the table scan operator on the 'store_sales' table, which accounts for approximately 79% of the execution time. It scans a total of 3,064,958,976 bytes and emits 54,704,849 output rows. The columns scanned are 'ss_sold_date_sk', 'ss_customer_sk', 'ss_store_sk', 'ss_quantity', 'ss_sales_price', and 'ss_ext_discount_amt'.

**Recommendations**:
1. It is suggested to join the 'store_sales' table with the 'date_dim' table on 'ss_sold_date_sk = d_date_sk' and apply a filter for 'd_year = 2001' during the initial table scan. This would reduce the number of rows processed and avoid scanning the entire 'store_sales' table.
2. The common table expressions (CTEs) 'sales_2001', 'monthly_spend_increase', and 'spend_increase_percentage' should be merged into a single query block using conditional aggregation and window functions. This would reduce the number of scans a

In [28]:
# Diff two queries
previous_query = """
SELECT
    employee_id,
    department,
    salary,
    AVG(salary) OVER (PARTITION BY department) AS department_avg_salary,
    salary - AVG(salary) OVER (PARTITION BY department) AS diff_from_avg
FROM
    employees;
"""
current_query = """
SELECT
    employee_id,
    department,
    salary,
    MAX(salary) OVER (PARTITION BY department) AS department_max_salary,
    salary - AVG(salary) OVER (PARTITION BY department) AS diff_from_avg
FROM
    employees;
LIMIT 100;
"""
print(agent.chat(f"tell me difference between {previous_query} and {current_query}"))

The difference between the two queries is as follows:

1. The first query calculates the average salary for each department and the difference between each employee's salary and the department's average salary. It does not have a row limit.

2. The second query calculates the maximum salary for each department instead of the average salary. It also includes the difference between each employee's salary and the department's average salary, similar to the first query. Additionally, this query has a LIMIT clause, restricting the results to the first 100 rows.

In summary, the key differences are the use of MAX(salary) instead of AVG(salary) for the department salary comparison and the inclusion of a LIMIT clause in the second query.


In [29]:
# Describe dataset
print(agent.chat("Summarize the dataset"))

The database is a comprehensive collection of schemas covering various domains:

- **FLIGHT**: Contains information about airlines, airports, and flight details.
- **STUDENT_TRANSCRIPTS_TRACKING**: Manages student records, enrollment, and academic information, including courses, degree programs, departments, and semesters.
- **WORLD**: Provides data on cities, countries, populations, and languages spoken.
- **INFORMATION_SCHEMA**: Offers metadata about the database objects within the WAII database, including roles, classes, columns, databases, functions, load history, privileges, procedures, constraints, schemata, sequences, and more.
- **EMPLOYEE_HIRE_EVALUATION**: Holds data on employee information, evaluations, hiring evaluations, and shop performance.
- **PETS**: Tracks the relationship between students and their pets, along with managing student records.

Each schema is designed to cater to specific data management needs, ranging from flight operations to student and employee reco

In [30]:
print(agent.chat("Give me questions which I can ask about this dataset"))

Here are some example questions you can ask about the different schemas within the dataset:

**For the FLIGHT schema:**
- What are the busiest airports in terms of flight departures?
- Which airline operates the most international routes?
- How many flights are there between two specific cities?

**For the STUDENT_TRANSCRIPTS_TRACKING schema:**
- Which students have the highest GPA in their respective departments?
- How many students are enrolled in each degree program?
- What is the average number of courses taken by students each semester?

**For the WORLD schema:**
- Which country has the highest population density?
- What are the official languages spoken in a specific country?
- How many countries have a population greater than 100 million?

**For the INFORMATION_SCHEMA schema:**
- What tables exist in a specific schema?
- Which columns in a table have constraints?
- How many stored procedures are defined in the database?

**For the EMPLOYEE_HIRE_EVALUATION schema:**
- Which emplo

In [31]:
q = """
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, lag, lead, round
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("yearly_car_analysis").getOrCreate()

yearly_avg_hp = cars_data.groupBy("year").agg(avg("horsepower").alias("avg_horsepower"))

windowSpec = Window.orderBy("year")

yearly_comparisons = yearly_avg_hp.select(
    "year",
    "avg_horsepower",
    lag("avg_horsepower").over(windowSpec).alias("prev_year_hp"),
    lead("avg_horsepower").over(windowSpec).alias("next_year_hp")
)

final_result = yearly_comparisons.select(
    "year",
    "avg_horsepower",
    round(
        (yearly_comparisons.avg_horsepower - yearly_comparisons.prev_year_hp) / 
        yearly_comparisons.prev_year_hp * 100, 2
    ).alias("percentage_diff_prev_year"),
    round(
        (yearly_comparisons.next_year_hp - yearly_comparisons.avg_horsepower) / 
        yearly_comparisons.avg_horsepower * 100, 2
    ).alias("percentage_diff_next_year")
).orderBy("year")

final_result.show()
"""
print(agent.chat(f"translate this pyspark query {q}, to Snowflake"))

The translated Snowflake SQL query from the given PySpark query is as follows:

```sql
WITH yearly_avg_hp AS (
    SELECT
        year,
        AVG(horsepower) AS avg_horsepower
    FROM waii.car.cars_data
    GROUP BY
        year
),

yearly_comparisons AS (
    SELECT
        year,
        avg_horsepower,
        LAG(avg_horsepower) OVER (ORDER BY year) AS prev_year_hp,
        LEAD(avg_horsepower) OVER (ORDER BY year) AS next_year_hp
    FROM yearly_avg_hp
)

SELECT
    year,
    avg_horsepower,
    ROUND((
        (
            avg_horsepower - prev_year_hp
        ) / NULLIF(prev_year_hp, 0) * 100
    ), 2) AS percentage_diff_prev_year,
    ROUND((
        (
            next_year_hp - avg_horsepower
        ) / NULLIF(avg_horsepower, 0) * 100
    ), 2) AS percentage_diff_next_year
FROM yearly_comparisons
ORDER BY
    year
```

This query calculates the average horsepower for each year, the previous year's average horsepower, and the next year's average horsepower. It then computes